customize dictionary for social media

In [ ]:
topics_hierarchy = {
    # --------------------------
    # 
    # --------------------------
    "International Affairs": {
        "Russia-Ukraine Conflict": {
    "keywords": ["Putin", "Zelensky", "Donbas", "Crimea", "Minsk Agreement", "Kharkiv"],
    "context_rules": [
        "region:Ukraine", 
        "region:Russia",
        "conflict_type:war"
    ],
    "exclude": ["COVID", "energy crisis"]  
    },
        "Israel-Gaza Conflict": {
            "keywords": ["Hamas", "IDF", "ceasefire", "settlements", "Gaza Strip"],
            "context_rules": ["region:Middle East"]
        },
        "Global Organizations": {
            "keywords": ["WHO", "UNICEF", "Paris Agreement", "Kyoto Protocol"],
            "context_rules": ["entity:international"]
        }
    },
    
    # --------------------------
    # 移民与边境（主主题）
    # --------------------------
    "Immigration & Border": {
        "Border Security": {
    "keywords": ["border wall", "illegal crossings", "ICE", "Border Patrol", "Border Security"],
    "exclude": ["Great Wall of China", "Berlin Wall", "Hadrian's Wall"],
    "context_rules": [
        "geo:US-Mexico|Texas|Arizona",
        "policy_scope:domestic"
    ]},
        "Citizenship Policies": {
            "keywords": ["DACA", "naturalization", "green card", "path to citizenship"],
            "context_rules": ["policy_type:legal"]
        }
    },
    
    # --------------------------
    # LGBTQ+ 议题（主主题）
    # --------------------------
    "LGBTQ+ Rights": {
        "Marriage & Relationships": {
            "keywords": ["same-sex marriage", "Obergefell v. Hodges", "civil unions"],
            "context_rules": ["law_type:supreme_court"]
        },
        "Transgender Rights": {
    "keywords": ["gender-affirming care", "bathroom laws", "HRT"],
    "exclude": ["general healthcare", "cosmetic surgery"],
    "context_rules": [
        "medical_context:gender_clinic",
        "policy_level:state|federal"
    ]},
        "Anti-Discrimination": {
            "keywords": ["Equality Act", "anti-LGBTQ laws", "discrimination"],
            "context_rules": ["policy_type:legislation"]
        }
    },
    
    # --------------------------
    # 经济（主主题）
    # --------------------------
    "Economy": {
        "Macroeconomic Indicators": {
    "keywords": ["GDP growth", "CPI", "unemployment rate", "BLS report"],
    "context_rules": [
        "data_source:official|BLS|Census Bureau",
        "frequency:quarterly"
    ]},
        "Trade Policy": {
            "keywords": ["tariffs", "USMCA", "WTO", "trade deficit"],
            "context_rules": ["policy_type:international"]
        },
        "Taxation": {
            "keywords": ["corporate tax", "IRS audits", "tax cuts"],
            "context_rules": ["policy_scope:federal"]
        }
    },
    
    # --------------------------
    # 社会议题（主主题）
    # --------------------------
    "Social Issues": {
        "Abortion Rights": {
            "keywords": ["Roe v. Wade", "fetal heartbeat bill", "Planned Parenthood"],
            "context_rules": ["policy_level:state"]
        },
        "Violent Crime": {
    "keywords": ["mass shootings", "gun violence", "homicide rates", "CDC firearm report"],
    "context_rules": [
        "data_source:FBI|CDC|local_agency",
        "policy_scope:state|municipal"
    ]}
    }
}

In [1]:
import pandas as pd
df = pd.read_parquet("detailed_posts_.parquet")

### few shoot prompting classification

In [3]:
import pandas as pd
import openai
import json
import time
import logging
from tenacity import retry, stop_after_attempt, wait_exponential
from tqdm import tqdm
from functools import lru_cache

In [21]:
import pandas as pd
import openai
import json
import logging
import asyncio
import re
from tenacity import retry, stop_after_attempt, wait_exponential
from tqdm import tqdm
from functools import lru_cache
from concurrent.futures import ThreadPoolExecutor

classification for socia l media only

In [ ]:
import os
import re
import asyncio
import pandas as pd
import logging
from functools import lru_cache
from tqdm.asyncio import tqdm
from tenacity import retry, stop_after_attempt, wait_exponential
from openai import AsyncOpenAI

# CONFIG
CONFIG = {
    "input_csv": "radio_stratified(in).csv",
    "output_csv": "classified_posts.csv",
    "text_column": "attributes.search_data_fields.all_text",
    "main_column": "main_category",
    "sub_column": "sub_category",
    "batch_size": 50,
    "max_parallel": 20,
    "cache_size": 5000,
    "model": "gpt-3.5-turbo",
    "temperature": 0.1,
    "max_retries": 3,
    "requests_per_minute": 5000,
    "resume": True
}

os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here" 
# 推荐通过环境变量设置

TOPICS = {
    "International Affairs": {
        "Russia-Ukraine Conflict": {
            "base_keywords": ["#StandWithUkraine", "PutinRegime", "NATOresponse", "WarCrimesAlert", "DonbasFrontline"],
            "exclude": ["COVID", "Pandemic"]
        },
        "Israel-Gaza Conflict": {
            "base_keywords": ["#FreePalestine", "IDFops", "HamasRockets", "CeasefireNOW", "SettlementsUpdate"],
            "exclude": []
        },
        "Global Organizations": {
            "base_keywords": ["UNvoteAlert", "WHOupdate", "ClimateAccord", "IMFwarning", "RefugeeAid"],
            "exclude": []
        }
    },
    "Immigration & Border": {
        "Border Security": {
            "base_keywords": ["#BorderCrisis", "MigrantCaravan", "ICEraids", "WallFunding", "AsylumBacklog"],
            "exclude": ["Great Wall"]
        },
        "Citizenship Policies": {
            "base_keywords": ["DACAupdate", "GreenCardLottery", "CitizenshipPath", "VisaOverhaul", "UndocumentedRights"],
            "exclude": []
        }
    },
    "LGBTQ+ Rights": {
        "Marriage & Relationships": {
            "base_keywords": ["#LoveIsLove", "MarriageEquality", "RainbowFamilies", "Obergefell", "SpousalBenefits"],
            "exclude": []
        },
        "Transgender Rights": {
            "base_keywords": ["#ProtectTransKids", "GenderCare", "BathroomBills", "YouthTransition", "HormoneAccess"],
            "exclude": ["general healthcare"]
        },
        "Anti-Discrimination": {
            "base_keywords": ["#EqualityActNow", "WorkplaceBias", "HateCrimeAlert", "SchoolPolicies", "LegalChallenges"],
            "exclude": []
        }
    },
    "Economy": {
        "Inflation": {
            "base_keywords": ["#InflationNation", "PriceHikeAlert", "Shrinkflation", "CPIspike", "WageLag"],
            "exclude": ["StockMarket", "Crypto"]
        },
        "Unemployment": {
            "base_keywords": ["#JobsReport", "LayoffWave", "TechLayoffs", "GigEconomy", "Underemployment"],
            "exclude": ["Retirement", "StudentJobs"]
        },
        "Economy Growth": {
            "base_keywords": ["GDPboost", "#EconomicRecovery", "ConsumerSpending", "StartupBubble", "RecessionWatch"],
            "exclude": ["PersonalFinance", "Cryptocurrency"]
        }
    },
    "Financial Markets": {
        "Stock Markets": {
            "base_keywords": ["#StockAlert", "MarketCrash", "EarningsSeason", "MemeStocks", "SP500update"],
            "exclude": []
        },
        "Crypto": {
            "base_keywords": ["#Bitcoin", "ETHsurge", "NFThype", "CryptoCrash", "DeFiTrends"],
            "exclude": []
        },
        "Commodities": {
            "base_keywords": ["OilPrice", "GoldRush", "CryptoMining", "AgriFutures", "EnergyCrisis"],
            "exclude": []
        }
    },
    "Social Issues": {
        "Abortion Rights": {
            "base_keywords": ["#RoeVWade", "AbortionAccess", "HeartbeatBill", "ClinicDefense", "StateBans"],
            "exclude": []
        },
        "Gun Control": {
            "base_keywords": ["#GunReformNow", "BGcheck", "RedFlagLaws", "MassShooting", "SchoolSafety"],
            "exclude": []
        }
    }
}

logging.basicConfig(
    filename='classification.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

class SocialMediaClassifier:
    def __init__(self):
        self.client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        self.semaphore = asyncio.Semaphore(CONFIG['max_parallel'])

    async def build_prompt(self, text: str) -> str:
        prompt = "Classify the following text into one of the specific categories below.\n"
        prompt += "Return the result as: Main Category | Sub Category\n\n"
        for main_cat, subcats in TOPICS.items():
            prompt += f"{main_cat}:\n"
            for subcat in subcats:
                prompt += f"  - {subcat}\n"
        prompt += f"\nText:\n{text[:1000]}\n\nOnly return: Main Category | Sub Category."
        return prompt

    @retry(stop=stop_after_attempt(CONFIG['max_retries']), wait=wait_exponential(min=2, max=10))
    async def classify_post(self, text: str) -> tuple[str, str]:
        try:
            prompt = await self.build_prompt(text)
            response = await self.client.chat.completions.create(
                model=CONFIG['model'],
                messages=[{"role": "user", "content": prompt}],
                temperature=CONFIG['temperature'],
                max_tokens=100
            )
            result = response.choices[0].message.content.strip()
            if "|" in result:
                main_cat, sub_cat = map(str.strip, result.split("|", 1))
                return main_cat, sub_cat
            return "error", "error"
        except Exception as e:
            logging.error(f"Classification failed for: {text[:100]} — {e}")
            return "error", "error"

class ProcessingPipeline:
    def __init__(self):
        self.classifier = SocialMediaClassifier()
        self.df = None

    async def load_data(self):
        self.df = pd.read_csv(CONFIG['input_csv'])
        for col in [CONFIG['main_column'], CONFIG['sub_column']]:
            if col not in self.df.columns:
                self.df[col] = None
        if CONFIG['resume']:
            self.df = self.df[self.df[CONFIG['main_column']].isna() | self.df[CONFIG['sub_column']].isna()]

    async def process_batch(self, texts):
        async def classify_fresh(text):
            async with self.classifier.semaphore:
                return await self.classifier.classify_post(text)
        return await asyncio.gather(*[classify_fresh(text) for text in texts])

    async def run(self):
        await self.load_data()
        total = len(self.df)

        with tqdm(total=total, desc="Classifying") as pbar:
            for i in range(0, total, CONFIG['batch_size']):
                batch = self.df.iloc[i:i + CONFIG['batch_size']]
                texts = batch[CONFIG['text_column']].fillna("").astype(str).tolist()
                results = await self.process_batch(texts)

                for j, (main_cat, sub_cat) in enumerate(results):
                    self.df.iat[i + j, self.df.columns.get_loc(CONFIG['main_column'])] = main_cat
                    self.df.iat[i + j, self.df.columns.get_loc(CONFIG['sub_column'])] = sub_cat

                pbar.update(len(results))

                if (i // CONFIG['batch_size']) % 2 == 0:
                    self.df.to_csv(CONFIG['output_csv'], index=False)

        self.df.to_csv(CONFIG['output_csv'], index=False)
        logging.info("Classification completed.")

if __name__ == "__main__":
    asyncio.run(ProcessingPipeline().run())


/var/folders/kg/jlb056ts6_x8ccsy2dk96hsr0000gn/T/ipykernel_27068/1461508114.py:157: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_csv(CONFIG['input_csv'])
Classifying: 100%|██████████| 35000/35000 [46:46<00:00, 12.47it/s]  
